In [10]:
import numpy as np

In [ ]:
class MapBelief():
    def __init__(self, init_map):
        self.map = init_map
        self.landmark_beliefs = [] # array of (mu, cov) pairs

    def likelihood(self, landmark):
        l = 0
        n = len(self.landmark_beliefs)
        id = 0
        max_like = -1
        for i, (mu, cov) in enumerate(self.landmark_beliefs):
            tmp = np.random.multivariate_normal_pdf(landmark, mu, cov)
            l += (1/n) * tmp
            # pick landmark with highest likelihood
            if tmp > max_like:
                id = i
                max_like = tmp

        return l, id
    
    def get_landmark_params(self, id):
        return self.landmark_beliefs[id]
    
    def set_landmark_params(self, id, params):
        self.landmark_beliefs[id] = params
    
    def add_landmark(self, params):
        self.landmark_beliefs.append(params)

    def draw_map(self):
        # how?
        pass

# Kalman Filter
def update_belief(mu, cov, z_t):
    # Update belief; estimate new state.
    C_t = np.eye(2)
    Q_t = np.array([[0.2, 0], 
                    [0, 0.2]])
    A_t = np.eye(2)
    R_t = np.eye(2) * 1e-1

    # prediction step. NOTE: assuming landmarks do not move
    mu_bar_next = (A_t @ mu)
    cov_bar_next = (A_t @ (cov @ A_t.T)) + R_t
    # kalman gain
    K_t = cov_bar_next @ (C_t.T @ np.linalg.inv((C_t @ (cov_bar_next @ C_t.T)) + Q_t))
    # correction step
    mu_next = mu_bar_next + (K_t @ (z_t - (C_t @ mu_bar_next)))
    cov_next = (np.eye(4) - (K_t @ C_t)) @ cov_bar_next

    return mu_next, cov_next
        
def map_update(z_t, map_belief: MapBelief):
    # update and add new observations
    # loop over all observed landmarks
    thresh = 0.5
    for obv in z_t:
        l, id = map_belief.likelihood(obv)
        if l > thresh:
            mu, cov = map_belief.get_landmark_params(id)
            # kalman update
            mu_new, cov_new = update_belief(mu, cov, obv)
            map_belief.set_landmark_params(id, (mu_new, cov_new))
        else:
            # add to map_belief.landmark_beliefs
            params = (obv, np.eye(2)*1e-1)
            map_belief.add_landmark(params)

    

# map_update is called everytime a robot gets observation
# how do we synchronize updates to global belief map
# first call first serve?


np.float64(0.37000000000000005)